In [9]:
import dlib
import cv2
import tensorflow as tf
import numpy as np
#import age_gender_test
from age_gender_model import * 

%load_ext autoreload
%autoreload 2

img_path = "./images/56_female.jpg"
model_path = "./train_models/age_gender_tensornets_wiki_cropface_model_20180525_180944"

#tf.reset_default_graph()
    
detector = dlib.get_frontal_face_detector() #获取人脸分类器

def add_margin(img, face_loc):
    crop_h = int(0.4 * (face_loc[3] - face_loc[1]))
    crop_w = int(0.4 * (face_loc[2] - face_loc[0]))
    img_h = img.shape[0]
    img_w = img.shape[1]
    #new_crop_low = crop_h + face_loc[1] - crop_h
    new_crop_high = crop_h + face_loc[3] + crop_h
    #new_crop_left = crop_w + face_loc[0] - crop_w
    new_crop_right = crop_w + face_loc[2] + crop_w
    replicate = cv2.copyMakeBorder(img, crop_h, crop_h, crop_w, crop_w, cv2.BORDER_REPLICATE)
    crop_face_img = replicate[face_loc[1] : new_crop_high, face_loc[0] : new_crop_right]
    return crop_face_img

tf.reset_default_graph()
with tf.Session() as sess:
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)

    print(img.shape)
    #b, g, r = cv2.split(img)    # 分离三个颜色通道
    #img = cv2.merge([r, g, b])   # 融合三个颜色通道生成新图片
    
    dets = detector(img, 1) #使用detector进行人脸检测 dets为返回的结果
    
    #input_tensor = tf.placeholder(tf.float32, [None, 224, 224, 3])
    HH, WW, _ = img.shape
    faces_data = np.zeros([len(dets), 224, 224, 3])
    print("dets: ", dets)
    for index, face in enumerate(dets):        
        # 在图片中标注人脸，并显示
        left = face.left()
        top = face.top()
        right = face.right()
        bottom = face.bottom()
        w = right - left
        h = bottom - top
        left = int(max(0, left - 0.4 * w))
        top = int(max(0, top - 0.4 * h))
        right = int(min(WW, right + 0.4 * w))
        bottom = int(min(HH, bottom + 0.4 * h))

        face_img = img[top:bottom, left:right].copy()
        cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 3)
        face_img = cv2.resize(face_img, (224, 224))
        cv2.imshow("?!!!!", face_img)
        faces_data[index] = face_img
        
        """        face_loc = np.array([162.616728383582,97.9220370301495,307.299783928807,242.605092575374])
                face_loc = (face_loc).astype("int32")
                face_img = add_margin(img, face_loc)
                face_img = cv2.resize(face_img, (224, 224))
                faces_data[index] = face_img"""
    #print("faces_data.shape", faces_data.shape)
    age, gender = run_test_temp(sess, faces_data, model_path)
    print("age: ", age)
    print("gender: ", gender)
    for i in range(len(dets)):
        #cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 3)
        font = cv2.FONT_HERSHEY_SIMPLEX
        if top - 20 < 0:
            top = top + 25
        else:
            top = top - 10
        gender_text = "male" if gender[i] == 1 else "female"
        cv2.putText(img,str(int(age[i])) + ", " + gender_text,(left,top), font, 1,(0,255,0),1,cv2.LINE_AA)
        cv2.imshow("test", img)
        #print("age: " + str(age[i]) + "gender: " + str(gender[i]))
cv2.waitKey()
cv2.destroyAllWindows()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(500, 431, 3)
dets:  rectangles[[(117, 118) (340, 341)]]
INFO:tensorflow:Restoring parameters from train_models/age_gender_tensornets_wiki_cropface_model_20180525_180944
age:  11
gender:  [0]


TypeError: 'int' object is not subscriptable

In [15]:
import dlib
import cv2
import numpy as np
import socket

%load_ext autoreload
%autoreload 2

img_path = "./images/28_female.jpg"
model_path = "./train_models/age_gender_tensornets_wiki_cropface_model_20180523_123604"

detector = dlib.get_frontal_face_detector() #获取人脸分类器
img = cv2.imread(img_path, cv2.IMREAD_COLOR)

print(img.shape)

address = ('35.227.146.91', 60000)  
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  

print(np.max(img))
print(np.min(img))
img = img.reshape([-1])
print(img.shape)
print(img[13])
msg = bytearray(img.shape[0])
for i in range(img.shape[0]):
    #t = int(img[i])
    #print(i)
    #p = t.to_bytes(1, 'big')
    msg[i] = img[i]
    #print(msg[i])
length_send = s.sendto(msg, address)
print(length_send)
s.close()  


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(268, 400, 3)
255
0
(321600,)
104


OSError: [WinError 10040] 一个在数据报套接字上发送的消息大于内部消息缓冲区或其他一些网络限制，或该用户用于接收数据报的缓冲区比数据报小。

In [ ]:


address = ('127.0.0.1', 31500)  
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  
s.bind(address)  

while True:  
    data, addr = s.recvfrom(2048)  
    if not data:  
        print("client has exist")
        break  
    print("received:", data, "from", addr)
  
s.close()  